In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import numpy as np
import os

Using TensorFlow backend


In [3]:
path_tales = 'bajki/'

texts = ''
for index, path in enumerate(os.listdir(path_tales)):
    text = ''
    with open(path_tales + path, encoding='utf-8') as f:
        text = f.read()
        texts += text

In [4]:
import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import re
import nltk
nltk.download('punkt')

def clean_text(txt):
    # Remove punctuation
    txt = "".join(v for v in txt if v not in string.punctuation and '\n' not in v)

    # Remove non-alphanumeric characters (including special characters)
    txt = re.sub(r'[^a-zA-Ząćęłńóśźż\s]', '', txt)

    # Convert to lowercase
    txt = txt.lower()

    return txt

path_tales = 'gdrive/MyDrive/bajki/'
sentences = []

for filename in tqdm.tqdm(os.listdir(path_tales)):
    with open(path_tales + filename, 'r', encoding='utf-8') as f:
        tale_text = f.read()
        tale_text = sent_tokenize(tale_text)

        sentences.extend([clean_text(l) for l in tale_text])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 85/85 [00:19<00:00,  4.37it/s]


In [5]:
sentences[:10]

['pewna uboga wdowa mieszkała w samotnej chatce a przed tą chatką był ogródek w którym rosły dwa krzaki róż jedne róże białe drugie czerwone',
 'wdowa miała dwie córeczki podobne do obu krzaków róż',
 'jedna zwała się białośnieżką a druga różanką',
 'dziewczątka były tak pobożne i dobre tak pracowite i roztropne że chyba drugich takich dzieci nie było na świecie ale białośnieżka była łagodniejszą niż różanka',
 'różanka wolała skakać po łąkach i polach rwała kwiatki i goniła ptaszyny leśne a białośnieżka siadywała w domu przy matce pomagała jej w gospodarstwie albo czytywała jej książki gdy nic innego nie było do roboty',
 'obie dziewczynki tak bardzo się kochały że zawsze idąc razem trzymały się za rączki a gdy białośnieżka mówiła my się nigdy nie rozstaniemy różanka odpowiadała tym co ma jedna musi się i druga podzielićnieraz biegały same po lesie i zbierały czerwone jagody żadne zwierzę nie robiło im nic złego ale przychodziły do nich z całym zaufaniem zajączek jadł z ich rąk kapust

In [6]:
import random
random.shuffle(sentences)

In [7]:
from collections import Counter

# Tokenize the sentences and count word frequencies
word_counts = Counter()
for sentence in sentences:
    words = sentence.split()
    word_counts.update(words)

# Select the top 50,000 most frequent words
top_words = set(word for word, _ in word_counts.most_common(50000))

# Replace the sentences with only the top words
new_sentences = []
for sentence in sentences:
    words = sentence.split()
    selected_words = [word for word in words if word in top_words]
    new_sentences.append(" ".join(selected_words))

In [8]:
new_sentences[:10]

['nad ludźmi odparła',
 'wtem pies nozdrza zapuścił w dół powąchał mocno podniósł się i szczeknął jakby w głąb jamy pokazywał chłopcu',
 'to niepodobna mruczał scrooge żebym przespał dzień i część drugiej nocy',
 'widzę',
 'ho',
 'jesteśmy',
 'postanowił być dumny do końca',
 'to okropne myślał przeprowadzać operację w takich warunkach',
 'natychmiast do powrotu wszelka chęć wszyscy wołają wojny chęć w nich pała',
 'ujrzałem światło i zaszedłem tutaj aż pod tę chałupę']

In [10]:
from tensorflow.keras.layers import TextVectorization

def custom_standardization(input_string):
    sentence = tf.strings.lower(input_string)
    sentence = tf.strings.regex_replace(sentence, "\n", " ")
    return sentence

maxlen = 50

vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

vectorize_layer.adapt(new_sentences)
vocab = vectorize_layer.get_vocabulary()
index_lookup = dict(zip(range(len(vocab)), vocab))

In [15]:
from tensorflow.keras.models import load_model
model_filename = 'transformer'  # Example filename for epoch 1

exp = load_model(model_filename)

In [22]:
def sample_token(logits):
    logits, indices = tf.math.top_k(logits, k=5, sorted=True)
    indices = np.asarray(indices).astype("int32")
    preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
    preds = np.asarray(preds).astype("float32")
    return np.random.choice(indices, p=preds)

decoded_sample = 'dawno dawno temu'

for i in range(50):
    tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
    predictions = exp.predict([tokenized_prompt], verbose=0)

    sample_index = len(decoded_sample.strip().split())-1

    sampled_token = sample_token(predictions[0][sample_index])
    sampled_token = index_lookup[sampled_token]
    decoded_sample += " " + sampled_token

print(f"\nSample text:\n{decoded_sample}...\n")


Sample text:
dawno dawno temu nie mogę powiedzieć ale i ja mam  do tego wszystkiego   co   innego do tego uczynić bo i nie mam  czasu i w tej porze planów zapowiedział że go nie ma do stracenia ale w pamięci   i że jest  dla niej i...

